# Importación de librerias

In [1]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

C:\Users\Jose\Anaconda3\envs\DeepLearning\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
tf.__version__

'2.1.0'

# Contrucción de la red neuronal

In [3]:
# Construimos el agente inteligente AI_Trader

class AI_Trader():
    def __init__(self, state_size, action_space=3, model_name="AITrader"):

        #state_size: tamaños de los estados del agente

        #action_space: numero de acciones que el agente podrá tomar,
        # por defecto serán tres acciones: "Manten", "Compra" y "Vende"

        #model_name: Nombre del modelo para identificarlo

        self.state_size = state_size
        self.action_space = action_space
        self.model_name = model_name

        #deque: se encarga de crear el batch para simular la experiencia de repeticion
        # es el encargado de hacer el experience-replay
        #maxlen le indicará el largo de acciones que puede almacenar
        # Se usa deque con un maxlen para no sobrecargar la memoría de la maquina
        # ya que deque está optimizado para estos casos

        self.memory = deque(maxlen=2000)

        # En el inventario se guardara todo lo que se ha ido comprando
        self.inventory = []

        # gamma: factor de aprendizaje de Vellman
        # al ratio epsilon lo definiremos como dinamico, ya que irá
        # cayendo a medida del entrenamiento.
        # al principio epsilon será de 1, donde siempre al principio
        # se querra que se tome una decisión aleatoria
        # este ratio caerá hasta un limite del 0.01

        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_final = 0.01
        self.epsilon_decay = 0.995

        self.model = self.model_builder()
    
    def model_builder(self):
    
        model = tf.keras.models.Sequential()

        model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))

        model.add(tf.keras.layers.Dense(units=64, activation='relu'))

        model.add(tf.keras.layers.Dense(units=128, activation='relu'))

        model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))

        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=0.001))

        return model
  
    def trade(self, state):
    
        if random.random() <= self.epsilon:
            return random.randrange(self.action_space)
    
    # Si no tenemos que escojer un valor random
    # Le pedimos al modelo que prediga una acción
    # el modelo retornara un array de tres espacios 
    # array que vamos a recorrer con argmax
    # para que nos retorne el indice que tiene 
    # la de mayor calidad
    
    # self.model.predict(state) retorna => [[ 9.890427 10.058331  9.672999]]
    # por eso es necesario pasar actions como actions[0]
    
        actions = self.model.predict(state)

        return np.argmax(actions[0])
  
  
    def batch_train(self, batch_size):
    
        # batch_size : Tamaño del bloque de entrenamiento que se utilizará
        # batch_size suele ser 32 o 64 o 128 y así..
        
        batch = []
        # Deque encola los objetos. Los al final de la fila
        # Por eso en ese for se recorre con los últimos elementos
        for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
            batch.append(self.memory[i])
      
        # Cada uno de los objetos en memoria (batch)
        # tendrá state : estado del agente cuando ocurrio la acción
        # action : la acción que llevó acabo en ese momento
        # reward : la recompenza que obtuvo
        # nex_state : cúal fue el siguiente estado
        # done : Si se finalizo el ejercició en el bloque
        for state, action, reward, next_state, done in batch:
            reward = reward
            if not done:
                # Ecuación de vellman
                reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
        
            target = self.model.predict(state)
            target[0][action] = reward
      
            self.model.fit(state, target, epochs=1, verbose=0)
      
        if self.epsilon > self.epsilon_final:
            self.epsilon *= self.epsilon_decay

# Definición de funciones adicionales

In [4]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [5]:
# Función de formato de precio

def stocks_price_format(n):
    if n < 0:
        return "- $ {0:2f}".format(abs(n))
    else:
        return "$ {0:2f}".format(abs(n))

In [6]:
# Carga del dataset

def dataset_loader(stock_name):
    
    dataset = data_reader.DataReader(stock_name, data_source="yahoo")
    start_date = str(dataset.index[0]).split()[0]
    end_date = str(dataset.index[-1]).split()[0]
    
    close = dataset['Close']
    
    return close

In [7]:
# State creator

def state_creator(data, timestep, window_size):
    # data : es el dataset descargado previamente
    # timestep : Fecha en la que se quiere predecir el valor del mercado
    # window_size : determina cuantos días anteriores se quieren
    # considerar para predecir el estado de la fecha timestep
  
    starting_id = timestep - window_size + 1
  
    #Se tomas los valoreos de los días anteiores al de los de predecir
    # Si no se tienen todos los valores anteriores al socilitado 
    # se rellenan con el primer día tantas veces haga falta
    
    if starting_id >= 0:
        windowed_data = data[starting_id:timestep+1]
    else:
        windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])
    
    state = []
    for i in range(window_size - 1):
        state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
    return np.array([state])

In [8]:
# Cargar una divisa de mercado

stock_name = "AAPL"
data = dataset_loader(stock_name)

data.head()

Date
2015-04-20    127.599998
2015-04-21    126.910004
2015-04-22    128.619995
2015-04-23    129.669998
2015-04-24    130.279999
Name: Close, dtype: float64

# Entrenar la IA Trader

### Configurar los paremetros

In [9]:
# window_size : 10 días anteriores desde el día que se quiere predecir

window_size = 10

#episodes : son epochs,cuantas veces se recorrerá todo el entorno

#episodes = 1000
episodes = 2

# batch_size : despues de 32 muestras, se actualizarán los pesos
batch_size = 32

# data_samples : datos de muestras, se define con menos 1 ya que el último
# parametro no tiene datos historicos para predecirse

data_samples = len(data) - 1

### Definir el modelo del AI Trader

In [10]:
trader = AI_Trader(window_size)

In [11]:
trader.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                352       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


### Bucle de entrenamiento

In [ ]:
for episode in range(1, episodes + 1):
  
    print("Episodio: {}/{}".format(episode, episodes))
  
    # Se define el estado inicial siempre es el mismo
    # Segun los datos del dataset, a partir del día 0
    # Con la ventana de días historicos a tomar window_size + 1
    # el más 1 para que se tome el día que se quiere predecir
    
    state = state_creator(data, 0, window_size + 1)
  
    # total_profit : Capital de la cuenta inicial es 0
    total_profit = 0
    
    # trader.inventory : Es el historial de todo lo que hemos ido comprando 
    # en el episodio

    trader.inventory = []
  
    # para cada episodio se recorrerá todo el dataset
    
    for t in tqdm(range(data_samples)):
    
        # Se entrená con el estado actual
        
        action = trader.trade(state)
    
        # se define proximo estado 
        
        next_state = state_creator(data, t+1, window_size + 1)
        reward = 0
    
        if action == 1: #Compra
            trader.inventory.append(data[t])
            print("AI Trader compró: ", stocks_price_format(data[t]))
      
        # Si tenemos acciones en el inventario podremos vender
        
        elif action == 2 and len(trader.inventory) > 0: #Vende
            buy_price = trader.inventory.pop(0)
      
            # Con max las recomenzas siempre serán positivas
        
            reward = max(data[t] - buy_price, 0)
            total_profit += data[t] - buy_price
            print("AI Trader vendió: ", stocks_price_format(data[t]), " Beneficio: " + stocks_price_format(data[t] - buy_price) )
      
        # si t == data_samples - 1 significa que se está en la ultima fecha
        # que se puede predecir
        
        if t == data_samples - 1:
            done = True
        else:
            done = False
      
        trader.memory.append((state, action, reward, next_state, done))
    
        state = next_state
    
        if done:
            print("########################")
            print("BENEFICIO TOTAL: {}".format(total_profit))
            print("########################")
    
        # Se el trader memory pide más memoria que el batch_size
        # o que es lo mismo si se tiene más memoria de la que se puede procesar
        # significa que hay que actualizar los pesos de la red neuronal
    
        if len(trader.memory) > batch_size:
            trader.batch_train(batch_size)
      
    # Cada dies episodios se guardara el modelo en un fichero
    
    if episode % 10 == 0:
        trader.model.save("ai_trader_{}.h5".format(episode))
    

  0%|          | 0/1258 [00:00<?, ?it/s]

Episodio: 1/2
AI Trader compró:  $ 128.619995
AI Trader compró:  $ 129.669998
AI Trader vendió:  $ 132.649994  Beneficio: $ 4.029999
AI Trader vendió:  $ 128.949997  Beneficio: - $ 0.720001
AI Trader compró:  $ 125.800003
AI Trader vendió:  $ 126.320000  Beneficio: $ 0.519997
AI Trader compró:  $ 125.870003
AI Trader compró:  $ 128.949997
AI Trader compró:  $ 130.070007
AI Trader vendió:  $ 130.059998  Beneficio: $ 4.189995
AI Trader vendió:  $ 131.389999  Beneficio: $ 2.440002
AI Trader compró:  $ 132.539993
AI Trader compró:  $ 132.039993
AI Trader compró:  $ 129.960007
AI Trader vendió:  $ 130.119995  Beneficio: $ 0.049988


  3%|▎         | 33/1258 [00:05<03:18,  6.18it/s]

AI Trader vendió:  $ 128.649994  Beneficio: - $ 3.889999


  3%|▎         | 34/1258 [00:09<27:36,  1.35s/it]

...

AI Trader compró:  $ 111.570000


 33%|███▎      | 410/1258 [26:32<1:01:35,  4.36s/it]

AI Trader compró:  $ 109.489998


 33%|███▎      | 411/1258 [26:37<1:02:12,  4.41s/it]

AI Trader vendió:  $ 109.900002  Beneficio: - $ 7.159996
